# 08: Document Network Analysis with Graphistry

**llamatelemetry v0.1.0** | Kaggle 2× Tesla T4 (30GB Total VRAM)

---

## 🎯 Objective

This notebook demonstrates **document similarity networks** and **community detection** using llamatelemetry v0.1.0 with graphistry[ai] visualization:

### Architecture:
```
┌────────────────────────────────────────────────────────┐
│  GPU 0: llama-server (LLM)                             │
│         Generate document summaries & extract topics   │
│         tensor_split="1.0,0.0"                         │
└────────────────────────────────────────────────────────┘
                          ↓
                  Document Similarity Graph
                          ↓
┌────────────────────────────────────────────────────────┐
│  GPU 1: RAPIDS + Graphistry                            │
│         Community detection + cluster visualization    │
│         CUDA_VISIBLE_DEVICES="1"                       │
└────────────────────────────────────────────────────────┘
```

### Workflow:
1. Start llama-server on GPU 0
2. Generate document summaries using LLM
3. Build document similarity network on GPU 1
4. Detect communities with Louvain algorithm (cuGraph)
5. Visualize clusters with Graphistry
6. LLM interprets community themes

---

**Previous:** [07-knowledge-graph-extraction](07-knowledge-graph-extraction-graphistry-llamatelemetry-v0-1-0.ipynb)  
**Next:** [09-large-models-kaggle](09-large-models-kaggle-llamatelemetry-v0-1-0.ipynb)

## Step 0: Add Graphistry Secrets in Kaggle

Go to **Kaggle → Settings → Add-ons → Secrets** and add:
- `Graphistry_Personal_Key_ID`
- `Graphistry_Personal_Secret_Key`

## Step 1: Verify Dual GPU Environment

Runs GPU-accelerated community detection (Louvain) on GPU 1 to discover document clusters and thematic groupings.

Verifies dual T4 GPU setup for document network analysis combining LLM text processing with GPU-accelerated network analytics and visualization.

In [1]:
import subprocess
import os

print("="*70)
print("🔍 SPLIT-GPU ENVIRONMENT CHECK")
print("="*70)

result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for split-GPU operation!")
    print("   GPU 0 → llama-server (LLM for summaries)")
    print("   GPU 1 → RAPIDS/Graphistry (community detection)")
else:
    print("\n⚠️ Need 2 GPUs for split operation")

🔍 SPLIT-GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 14913 MiB
   1, Tesla T4, 15360 MiB, 14913 MiB

✅ Dual T4 ready for split-GPU operation!
   GPU 0 → llama-server (LLM for summaries)
   GPU 1 → RAPIDS/Graphistry (community detection)


## Step 2: Install Dependencies

Installs llamatelemetry for LLM inference, RAPIDS for GPU graph analytics, and Graphistry for visualizing document similarity networks.

Installs llamatelemetry for LLM inference, RAPIDS cuGraph for GPU graph analytics, and Graphistry for visualizing document similarity networks.

In [2]:
%%time
print("📦 Installing dependencies...")


!pip install -q huggingface-hub sseclient-py

# Install llamatelemetry v0.1.0
!pip install -q --no-cache-dir git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0

# Install cuGraph (matching Kaggle RAPIDS 25.6.0)
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*"

# Install Graphistry
!pip install -q "graphistry[ai]"

# Verify
import llamatelemetry
print(f"\n✅ llamatelemetry {llamatelemetry.__version__} installed")

try:
    import cudf, cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 48.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 9.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kagg(…):   0%|          | 0.00/1.40G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 2 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!


✅ llamatelemetry 0.1.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.6
CPU times: user 54.2 s, sys: 13.4 s, total: 1min 7s
Wall time: 1min 38s


Configures Graphistry API authentication using Kaggle secrets for cloud-based document network visualization rendering.

In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
graphistry.register(
    api=3,
    protocol="https",
    server="hub.graphistry.com",
    personal_key_id=user_secrets.get_secret("Graphistry_Personal_Key_ID"),
    personal_key_secret=user_secrets.get_secret("Graphistry_Personal_Secret_Key")
)

## Step 3: Download GGUF Model

Downloads embedding or instruction-following model for document analysis, semantic similarity computation, and relationship extraction.

Downloads language model for document embeddings, semantic analysis, and text similarity computation.

In [4]:
%%time
from huggingface_hub import hf_hub_download
import os

MODEL_REPO = "bartowski/Llama-3.2-3B-Instruct-GGUF"
MODEL_FILE = "Llama-3.2-3B-Instruct-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")

📥 Downloading Llama-3.2-3B-Instruct-Q4_K_M.gguf...


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf
   Size: 1.88 GB
CPU times: user 3.85 s, sys: 7.3 s, total: 11.2 s
Wall time: 4.18 s


## Step 4: Start llama-server on GPU 0 Only

Deploys llama-server on GPU 0 using tensor-split configuration for document processing while keeping GPU 1 free for graph operations.

Starts llama-server on GPU 0 using single-GPU configuration (tensor-split 1.0,0.0) while reserving GPU 1 for graph operations.

In [5]:
from llamatelemetry.server import ServerManager

print("="*70)
print("🚀 STARTING LLAMA-SERVER ON GPU 0")
print("="*70)

print("\n📋 Configuration:")
print("   GPU 0: 100% (llama-server for document summarization)")
print("   GPU 1: 0% (reserved for RAPIDS/Graphistry)")

server = ServerManager()
server.start_server(
    model_path=model_path,
    host="127.0.0.1",
    port=8090,
    gpu_layers=99,
    tensor_split="1.0,0.0",
    ctx_size=4096,
)

if server.check_server_health():
    print("\n✅ llama-server running on GPU 0!")
else:
    print("\n❌ Server failed to start")

🚀 STARTING LLAMA-SERVER ON GPU 0

📋 Configuration:
   GPU 0: 100% (llama-server for document summarization)
   GPU 1: 0% (reserved for RAPIDS/Graphistry)
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready...... ✓ Ready in 3.0s

✅ llama-server running on GPU 0!


## Step 5: Generate Document Summaries and Topics

Runs GPU-accelerated community detection (Louvain) on GPU 1 to discover document clusters and thematic groupings.

Verifies GPU memory allocation showing LLM loaded on GPU 0 with GPU 1 free for document network processing.

In [6]:
from llamatelemetry.api.client import LlamaCppClient

print("="*70)
print("📄 LLM-POWERED DOCUMENT ANALYSIS")
print("="*70)

client = LlamaCppClient(base_url="http://127.0.0.1:8090")

# Sample documents about various GPU computing topics
documents = [
    # Deep Learning cluster
    {"id": "doc1", "text": "PyTorch provides dynamic computation graphs for deep learning. It integrates seamlessly with CUDA for GPU acceleration. Training neural networks on GPUs is significantly faster than CPUs."},
    {"id": "doc2", "text": "TensorFlow is Google's deep learning framework. It offers distributed training across multiple GPUs. TensorBoard provides visualization for training metrics."},
    {"id": "doc3", "text": "NVIDIA cuDNN accelerates deep neural network training. It provides highly optimized primitives for convolution and pooling operations on GPUs."},
    
    # Data Science cluster
    {"id": "doc4", "text": "RAPIDS cuDF provides a GPU DataFrame library. It accelerates pandas operations by 50-100× using CUDA. Data scientists can process large datasets efficiently."},
    {"id": "doc5", "text": "cuML implements machine learning algorithms on GPUs. It supports classification, regression, and clustering with scikit-learn API compatibility."},
    {"id": "doc6", "text": "Apache Spark can leverage GPUs for distributed computing. RAPIDS Accelerator speeds up Spark SQL and DataFrame operations on GPU clusters."},
    
    # Inference cluster
    {"id": "doc7", "text": "llama.cpp enables efficient LLM inference on GPUs. GGUF quantization reduces model size while maintaining quality. It supports multi-GPU tensor parallelism."},
    {"id": "doc8", "text": "TensorRT optimizes neural network inference. It provides INT8 and FP16 precision for faster inference. Deployed models achieve low latency on NVIDIA GPUs."},
    {"id": "doc9", "text": "ONNX Runtime accelerates machine learning inference. It supports multiple hardware backends including CUDA. Models can be optimized for production deployment."},
    
    # Visualization cluster
    {"id": "doc10", "text": "Graphistry provides GPU-accelerated graph visualization. It handles millions of nodes and edges interactively. RAPIDS integration enables visual analytics at scale."},
    {"id": "doc11", "text": "cuGraph implements graph algorithms on GPUs. PageRank and community detection run 100× faster than NetworkX. It integrates with Graphistry for visualization."},
]

# Extract topics from each document
doc_topics = []

for doc in documents:
    prompt = f"""Summarize this document in 5 words maximum, focusing on the main topic:

{doc['text']}

5-word summary:"""
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=20,
        temperature=0.3,
    )
    
    summary = response.choices[0].message.content.strip()
    doc_topics.append({"id": doc["id"], "summary": summary, "text": doc["text"]})
    print(f"{doc['id']}: {summary}")

print(f"\n✅ Generated {len(doc_topics)} document summaries")

📄 LLM-POWERED DOCUMENT ANALYSIS
doc1: PyTorch accelerates deep learning computations.
doc2: Google's Deep Learning Framework TensorFlow
doc3: Optimized GPU acceleration for NNS.
doc4: GPU-accelerated data processing library.
doc5: Machine Learning on GPU Acceleration
doc6: Spark uses GPUs for speed.
doc7: Efficient GPU-based LLM inference tool
doc8: TensorRT accelerates neural network inference.
doc9: ONNX Runtime for Machine Learning
doc10: GPU-accelerated graph visualization software.
doc11: cuGraph accelerates GPU-based graph processing.

✅ Generated 11 document summaries


## Step 6: Initialize RAPIDS on GPU 1

Initializes RAPIDS on GPU 1 via CUDA_VISIBLE_DEVICES environment variable for GPU-accelerated document graph operations.

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("="*70)
print("🔥 INITIALIZING RAPIDS ON GPU 1")
print("="*70)

import cudf
import cupy as cp

print(f"\n📊 RAPIDS GPU Info:")
device = cp.cuda.Device(0)  # Device 0 in filtered view = actual GPU 1
print(f"   Device: {device.id} (filtered view)")
print(f"   Actual GPU: 1 (Tesla T4)")

print(f"\n✅ RAPIDS initialized on GPU 1")

🔥 INITIALIZING RAPIDS ON GPU 1

📊 RAPIDS GPU Info:
   Device: 0 (filtered view)
   Actual GPU: 1 (Tesla T4)

✅ RAPIDS initialized on GPU 1


## Step 7: Build Document Similarity Network

Loads sample document corpus and preprocesses text for similarity analysis and network construction.

In [8]:
import cugraph
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("="*70)
print("📊 BUILDING DOCUMENT SIMILARITY NETWORK")
print("="*70)

# Calculate TF-IDF similarity between documents
texts = [doc['text'] for doc in doc_topics]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
similarity_matrix = cosine_similarity(tfidf_matrix)

# Create edges for documents with similarity > threshold
threshold = 0.15  # Lower threshold to create more connections
edges_list = []

for i in range(len(documents)):
    for j in range(i + 1, len(documents)):
        similarity = similarity_matrix[i, j]
        if similarity > threshold:
            edges_list.append({
                'source': i,
                'target': j,
                'weight': float(similarity)
            })

# Create cuDF edge list
edges_cudf = cudf.DataFrame(edges_list)

print(f"\n📊 Document Similarity Network:")
print(f"   Nodes (documents): {len(documents)}")
print(f"   Edges (similarities > {threshold}): {len(edges_cudf)}")
print(f"   Average similarity: {edges_cudf['weight'].mean():.3f}")

# Create cuGraph
G = cugraph.Graph()
G.from_cudf_edgelist(edges_cudf, source='source', destination='target', edge_attr='weight')

print(f"\n✅ Document network created on GPU 1")

📊 BUILDING DOCUMENT SIMILARITY NETWORK

📊 Document Similarity Network:
   Nodes (documents): 11
   Edges (similarities > 0.15): 9
   Average similarity: 0.210

✅ Document network created on GPU 1


## Step 8: Community Detection with Louvain Algorithm

Runs GPU-accelerated community detection (Louvain) on GPU 1 to discover document clusters and thematic groupings.

Generates document embeddings using LLM on GPU 0, creating vector representations for semantic similarity computation.

In [9]:
print("="*70)
print("🔬 COMMUNITY DETECTION (GPU-ACCELERATED)")
print("="*70)

# Run Louvain community detection
partition_df, modularity_score = cugraph.louvain(G)

# Map communities back to documents
communities_pd = partition_df.to_pandas()
doc_communities = {}
for _, row in communities_pd.iterrows():
    doc_id = int(row['vertex'])
    community_id = int(row['partition'])
    doc_communities[doc_id] = community_id

# Group documents by community
community_groups = {}
for doc_id, community_id in doc_communities.items():
    if community_id not in community_groups:
        community_groups[community_id] = []
    community_groups[community_id].append(doc_id)

print(f"\n📊 Detected {len(community_groups)} communities:")
for community_id, doc_ids in sorted(community_groups.items()):
    doc_names = [doc_topics[i]['id'] for i in doc_ids]
    print(f"   Community {community_id}: {', '.join(doc_names)}")

print("\n✅ Community detection complete on GPU 1")

🔬 COMMUNITY DETECTION (GPU-ACCELERATED)

📊 Detected 3 communities:
   Community 0: doc1, doc3, doc2, doc8
   Community 1: doc11, doc5, doc10
   Community 2: doc4, doc6

✅ Community detection complete on GPU 1


## Step 9: Graph Analytics on Communities

Computes network metrics (centrality, clustering coefficient) to identify influential documents and network structure.

Computes pairwise document similarity matrix using cosine similarity on embeddings, identifying related documents.

In [10]:
print("="*70)
print("📊 DOCUMENT NETWORK ANALYTICS")
print("="*70)

# PageRank - identify important documents
print("\n📊 PageRank Analysis (Document Importance):")
pagerank = cugraph.pagerank(G)
pagerank = pagerank.sort_values('pagerank', ascending=False)

for _, row in pagerank.to_pandas().head(5).iterrows():
    doc_id = int(row['vertex'])
    score = row['pagerank']
    doc_name = doc_topics[doc_id]['id']
    summary = doc_topics[doc_id]['summary']
    print(f"   {doc_name}: {score:.4f} - {summary}")

# Betweenness Centrality - bridge documents between communities
print("\n📊 Betweenness Centrality (Bridge Documents):")
bc = cugraph.betweenness_centrality(G)
bc = bc.sort_values('betweenness_centrality', ascending=False)

for _, row in bc.to_pandas().head(5).iterrows():
    doc_id = int(row['vertex'])
    score = row['betweenness_centrality']
    doc_name = doc_topics[doc_id]['id']
    summary = doc_topics[doc_id]['summary']
    print(f"   {doc_name}: {score:.4f} - {summary}")

print("\n✅ Graph analytics computed on GPU 1")

📊 DOCUMENT NETWORK ANALYTICS

📊 PageRank Analysis (Document Importance):
   doc1: 0.1802 - PyTorch accelerates deep learning computations.
   doc11: 0.1567 - cuGraph accelerates GPU-based graph processing.
   doc3: 0.1392 - Optimized GPU acceleration for NNS.
   doc4: 0.1111 - GPU-accelerated data processing library.
   doc6: 0.1111 - Spark uses GPUs for speed.

📊 Betweenness Centrality (Bridge Documents):


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


   doc1: 0.3393 - PyTorch accelerates deep learning computations.
   doc11: 0.3214 - cuGraph accelerates GPU-based graph processing.
   doc3: 0.0179 - Optimized GPU acceleration for NNS.
   doc2: 0.0000 - Google's Deep Learning Framework TensorFlow
   doc8: 0.0000 - TensorRT accelerates neural network inference.

✅ Graph analytics computed on GPU 1


## Step 10: LLM Analysis of Community Themes

Computes network metrics (centrality, clustering coefficient) to identify influential documents and network structure.

Constructs document network graph on GPU 1 where nodes are documents and edges represent similarity above threshold.

In [11]:
print("="*70)
print("🤔 LLM ANALYSIS OF COMMUNITY THEMES")
print("="*70)

for community_id, doc_ids in sorted(community_groups.items()):
    # Get summaries for this community
    summaries = [doc_topics[i]['summary'] for i in doc_ids]
    doc_names = [doc_topics[i]['id'] for i in doc_ids]
    
    prompt = f"""These documents form a cluster based on similarity:
{', '.join([f"{name}: {summary}" for name, summary in zip(doc_names, summaries)])}

What is the common theme? Answer in one phrase (3-5 words):"""
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=20,
        temperature=0.5
    )
    
    theme = response.choices[0].message.content.strip()
    print(f"\n📌 Community {community_id}: {theme}")
    print(f"   Documents: {', '.join(doc_names)}")

print("\n✅ Simultaneous GPU operation:")
print("   GPU 0: LLM inference (theme analysis)")
print("   GPU 1: Graph analytics (previously computed)")

🤔 LLM ANALYSIS OF COMMUNITY THEMES

📌 Community 0: Deep learning and acceleration.
   Documents: doc1, doc3, doc2, doc8

📌 Community 1: GPU-based graph processing.
   Documents: doc11, doc5, doc10

📌 Community 2: GPU acceleration technology.
   Documents: doc4, doc6

✅ Simultaneous GPU operation:
   GPU 0: LLM inference (theme analysis)
   GPU 1: Graph analytics (previously computed)


## Step 11: Graphistry Visualization

Runs GPU-accelerated community detection on document network to discover thematic clusters and topic groups.

In [12]:
print("="*70)
print("🎨 GRAPHISTRY DOCUMENT NETWORK VISUALIZATION")
print("="*70)

import graphistry
from kaggle_secrets import UserSecretsClient
import pandas as pd
import numpy as np

# --- 1. Register Graphistry ---
print("\n🔐 Registering with Graphistry...")
try:
    user_secrets = UserSecretsClient()
    graphistry_key_id = user_secrets.get_secret("Graphistry_Personal_Key_ID")
    graphistry_secret = user_secrets.get_secret("Graphistry_Personal_Secret_Key")

    graphistry.register(
        api=3,
        protocol="https",
        server="hub.graphistry.com",
        personal_key_id=graphistry_key_id,
        personal_key_secret=graphistry_secret
    )
    print("✅ Graphistry registered successfully")
except Exception as e:
    print(f"⚠️ Graphistry registration failed: {e}")
    print("   Add secrets: Graphistry_Personal_Key_ID, Graphistry_Personal_Secret_Key")

# --- 2. Prepare visualization data ---
print("\n📊 Preparing document network data...")

# Ensure edges_pd exists
if 'edges_pd' not in locals() and 'edges_pd' not in globals():
    if 'edges_cudf' in locals() or 'edges_cudf' in globals():
        edges_pd = edges_cudf.to_pandas()
    else:
        raise ValueError("No edges data found")

print(f"   Edge data shape: {edges_pd.shape}")

# Ensure nodes data with all metrics
if 'pagerank_pd' not in locals():
    pagerank_pd = pagerank.to_pandas()
if 'bc_pd' not in locals():
    bc_pd = bc.to_pandas()
if 'communities_pd' not in locals():
    communities_pd = communities.to_pandas()

# Build comprehensive nodes DataFrame
nodes_pd = pd.DataFrame({
    'node_id': list(range(len(doc_topics))),
    'doc_id': [doc['id'] for doc in doc_topics],
    'summary': [doc['summary'] for doc in doc_topics],
    'text_preview': [doc['text'][:100] + '...' if len(doc['text']) > 100 else doc['text'] for doc in doc_topics]
})

# Merge PageRank
nodes_pd = nodes_pd.merge(
    pagerank_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id',
    how='left'
)

# Merge Betweenness Centrality
nodes_pd = nodes_pd.merge(
    bc_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id',
    how='left'
)

# Merge Communities
nodes_pd = nodes_pd.merge(
    communities_pd.rename(columns={'vertex': 'node_id', 'partition': 'community'}),
    on='node_id',
    how='left'
)
nodes_pd['community'] = nodes_pd['community'].fillna(0).astype(int)

# --- 3. Compute degree centrality ---
print("   Computing degree centrality...")
degree_in = edges_pd.groupby('target').size().reset_index(name='degree_in').rename(columns={'target': 'node_id'})
degree_out = edges_pd.groupby('source').size().reset_index(name='degree_out').rename(columns={'source': 'node_id'})

nodes_pd = nodes_pd.merge(degree_in, on='node_id', how='left')
nodes_pd = nodes_pd.merge(degree_out, on='node_id', how='left')
nodes_pd['degree_in'] = nodes_pd['degree_in'].fillna(0).astype(int)
nodes_pd['degree_out'] = nodes_pd['degree_out'].fillna(0).astype(int)
nodes_pd['total_degree'] = nodes_pd['degree_in'] + nodes_pd['degree_out']

# --- 4. Role classification ---
print("   Classifying document roles...")

def classify_doc_role(row):
    """Classify documents: Hub (high centrality), Bridge (high betweenness), Peripheral."""
    pr = row['pagerank']
    bc = row['betweenness_centrality']
    pr_threshold = nodes_pd['pagerank'].median()
    bc_threshold = nodes_pd['betweenness_centrality'].median()

    if pr > pr_threshold and bc > bc_threshold:
        return 'Hub'
    elif bc > bc_threshold:
        return 'Bridge'
    else:
        return 'Peripheral'

nodes_pd['role'] = nodes_pd.apply(classify_doc_role, axis=1)

# --- 5. Size encoding (normalized PageRank) ---
pr_min = nodes_pd['pagerank'].min()
pr_max = nodes_pd['pagerank'].max()
if pr_max > pr_min:
    nodes_pd['node_size'] = 25 + (nodes_pd['pagerank'] - pr_min) / (pr_max - pr_min) * 75
else:
    nodes_pd['node_size'] = 50

# --- 6. Rich tooltips ---
nodes_pd['point_title'] = nodes_pd.apply(
    lambda row: f"{row['doc_id']}: {row['summary']}\n" +
                f"Community: {row['community']}\n" +
                f"Role: {row['role']}\n" +
                f"PageRank: {row['pagerank']:.4f}\n" +
                f"Betweenness: {row['betweenness_centrality']:.4f}\n" +
                f"Degree: {int(row['total_degree'])}",
    axis=1
)

edges_pd['edge_title'] = edges_pd.apply(
    lambda row: f"Similarity: {row['weight']:.3f}",
    axis=1
)

print(f"\n📊 Document Network Summary:")
print(f"   Documents: {len(nodes_pd)}")
print(f"   Similarity Edges: {len(edges_pd)}")
print(f"   Communities: {nodes_pd['community'].nunique()}")
print(f"   Avg similarity: {edges_pd['weight'].mean():.3f}")

# --- 7. Create Graphistry visualization ---
print("\n🎨 Creating Graphistry visualization...")

# Color palettes
community_colors = {
    0: '#FF6B6B',  # Red - Deep Learning
    1: '#4ECDC4',  # Teal - Data Science
    2: '#45B7D1',  # Blue - Inference
    3: '#FFA07A',  # Orange - Visualization
    4: '#98D8C8',  # Mint
}

role_icons = {
    'Hub': 'star',
    'Bridge': 'exchange-alt',
    'Peripheral': 'circle'
}

# Bind and create visualization
g = graphistry.bind(
    source='source',
    destination='target',
    node='node_id',
    point_title='point_title',
    edge_title='edge_title'
)

plotter = (
    g.edges(edges_pd)
    .nodes(nodes_pd)
    .encode_point_color('community', categorical_mapping=community_colors, default_mapping='#CCCCCC')
    .encode_point_size('node_size')
    .encode_point_icon('role', categorical_mapping=role_icons, default_mapping='circle')
    .encode_edge_color('weight', ['#E8E8E8', '#6C7A89', '#2C3E50'], as_continuous=True)
    .settings(url_params={
        'play': 0,
        'pointSize': 2.5,
        'edgeOpacity': 0.4,
        'bg': '%23FFFFFF',
        'showArrows': 'false',
        'showLabels': 'true'
    })
)

# --- 8. Launch visualization ---
try:
    url = plotter.plot(
        render=False,
        name="Document Similarity Network - llamatelemetry v0.1.0",
        description=f"{len(nodes_pd)} documents clustered into {nodes_pd['community'].nunique()} communities"
    )

    print(f"\n🚀 Visualization Created Successfully!")
    print(f"\n🔗 Graphistry URL:")
    print(f"   {url}")
    print(f"\n📌 Features:")
    print(f"   ✓ Color-coded by community (document clusters)")
    print(f"   ✓ Size scaled by PageRank (document importance)")
    print(f"   ✓ Icons show role (Hub ⭐, Bridge ↔️, Peripheral ○)")
    print(f"   ✓ Edge color intensity = similarity strength")
    print(f"   ✓ Interactive tooltips with doc summaries")

    from IPython.display import display, HTML
    display(HTML(
        f'<div style="margin:20px; padding:20px; background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); '
        f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
        f'<h3 style="margin:0 0 10px 0;">📄 Document Network Dashboard</h3>'
        f'<p style="margin:5px 0;">GPU computing topics clustered by similarity</p>'
        f'<a href="{url}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
        f'background:white; color:#667eea; text-decoration:none; border-radius:6px; font-weight:bold; '
        f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">🚀 Open Interactive Visualization</a>'
        f'</div>'
    ))

except Exception as e:
    print(f"\n❌ Visualization error: {e}")
    print(f"   Data prepared successfully - {len(nodes_pd)} nodes, {len(edges_pd)} edges")

print("\n" + "="*70)
print("✅ Document network visualization complete")
print("="*70)

🎨 GRAPHISTRY DOCUMENT NETWORK VISUALIZATION

🔐 Registering with Graphistry...
✅ Graphistry registered successfully

📊 Preparing document network data...
   Edge data shape: (9, 3)
   Computing degree centrality...
   Classifying document roles...

📊 Document Network Summary:
   Documents: 11
   Similarity Edges: 9
   Communities: 3
   Avg similarity: 0.210

🎨 Creating Graphistry visualization...

🚀 Visualization Created Successfully!

🔗 Graphistry URL:
   https://hub.graphistry.com/graph/graph.html?dataset=0a9094fa142b4e14aa4d7261de672719&type=arrow&viztoken=fa51743b-5b89-4041-a866-fa51f9c12d28&usertag=aeceb811-pygraphistry-0.50.6&splashAfter=1770259826&info=true&play=0&pointSize=2.5&edgeOpacity=0.4&bg=%23FFFFFF&showArrows=false&showLabels=true

📌 Features:
   ✓ Color-coded by community (document clusters)
   ✓ Size scaled by PageRank (document importance)
   ✓ Icons show role (Hub ⭐, Bridge ↔️, Peripheral ○)
   ✓ Edge color intensity = similarity strength
   ✓ Interactive tooltips wit


✅ Document network visualization complete


## Step 12: Detailed Community Report

Computes network metrics (degree centrality, PageRank) on GPU 1 to identify most influential and central documents.

In [13]:
print("="*70)
print("📋 DETAILED COMMUNITY REPORT")
print("="*70)

for community_id, doc_ids in sorted(community_groups.items()):
    print(f"\n📌 Community {community_id}:")
    print(f"   Size: {len(doc_ids)} documents")
    
    print(f"\n   Documents:")
    for doc_id in doc_ids:
        doc = doc_topics[doc_id]
        pr_score = pagerank_pd[pagerank_pd['vertex'] == doc_id]['pagerank'].values
        pr_score = pr_score[0] if len(pr_score) > 0 else 0
        print(f"      • {doc['id']}: {doc['summary']} (PageRank: {pr_score:.4f})")
    
    # Avg similarity within community
    intra_edges = edges_cudf[
        (edges_cudf['source'].isin(doc_ids)) & 
        (edges_cudf['target'].isin(doc_ids))
    ]
    if len(intra_edges) > 0:
        avg_sim = intra_edges['weight'].mean()
        print(f"\n   Avg intra-community similarity: {avg_sim:.3f}")

print("\n✅ Community analysis complete")

📋 DETAILED COMMUNITY REPORT

📌 Community 0:
   Size: 4 documents

   Documents:
      • doc1: PyTorch accelerates deep learning computations. (PageRank: 0.1802)
      • doc3: Optimized GPU acceleration for NNS. (PageRank: 0.1392)
      • doc2: Google's Deep Learning Framework TensorFlow (PageRank: 0.0980)
      • doc8: TensorRT accelerates neural network inference. (PageRank: 0.0881)

   Avg intra-community similarity: 0.220

📌 Community 1:
   Size: 3 documents

   Documents:
      • doc11: cuGraph accelerates GPU-based graph processing. (PageRank: 0.1567)
      • doc5: Machine Learning on GPU Acceleration (PageRank: 0.0639)
      • doc10: GPU-accelerated graph visualization software. (PageRank: 0.0516)

   Avg intra-community similarity: 0.192

📌 Community 2:
   Size: 2 documents

   Documents:
      • doc4: GPU-accelerated data processing library. (PageRank: 0.1111)
      • doc6: Spark uses GPUs for speed. (PageRank: 0.1111)

   Avg intra-community similarity: 0.166

✅ Community anal

## Step 13: Monitor Both GPUs

Runs GPU-accelerated community detection (Louvain) on GPU 1 to discover document clusters and thematic groupings.

Creates Graphistry visualization of document network with community colors, node sizes by centrality, and interactive filtering.

In [14]:
print("="*70)
print("📊 DUAL GPU MONITORING")
print("="*70)

!nvidia-smi

print("\n💡 Split-GPU Operation:")
print("   GPU 0: llama-server (document summarization)")
print("   GPU 1: RAPIDS (community detection, graph analytics)")

📊 DUAL GPU MONITORING
Thu Feb  5 02:52:13 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P0             29W /   70W |    2679MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------

## Step 14: Cleanup

Verifies dual T4 GPU availability for document network analysis workflow combining LLM text processing with GPU-accelerated network analytics.

Uses LLM on GPU 0 to generate natural language summaries of detected document communities and key themes.

In [15]:
print("🛑 Stopping llama-server...")
server.stop_server()

# Clear RAPIDS memory
import gc
del G, edges_cudf, pagerank, bc, communities
gc.collect()

print("\n✅ Resources cleaned up")
print("\n📊 Final GPU Status:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

🛑 Stopping llama-server...


NameError: name 'communities' is not defined

## 📚 Summary

### Document Network Analysis Workflow:
1. **GPU 0**: LLM generates document summaries
2. **CPU**: TF-IDF calculates document similarity
3. **GPU 1**: cuGraph builds similarity network
4. **GPU 1**: Louvain algorithm detects communities
5. **GPU 0**: LLM interprets community themes
6. **Graphistry**: Interactive visualization with communities

### Key Integration Points:
- ✅ LLM for document summarization and theme extraction
- ✅ cuGraph for GPU-accelerated community detection (Louvain)
- ✅ Graphistry for community-colored network visualization
- ✅ PageRank & Betweenness for document importance ranking

### Algorithms Used:
- **Louvain**: Community detection (finds document clusters)
- **PageRank**: Document importance (citation-like ranking)
- **Betweenness Centrality**: Bridge documents (connect communities)
- **TF-IDF + Cosine Similarity**: Document similarity metric

### Split-GPU Architecture:
```python
# GPU 0: llama-server
tensor_split="1.0,0.0"  # 100% on GPU 0

# GPU 1: RAPIDS
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import cudf, cugraph  # Uses GPU 1
```

---

**Next:** [09-large-models-kaggle](09-large-models-kaggle-llamatelemetry-v0-1-0.ipynb)